In [ ]:
# Import all the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
import os
import re
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer
from get_tweets import get_tweets_by_search_term, get_tweets_by_username
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", None)

In [ ]:
# Import NLTK lexicon
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('twitter_samples')


In [ ]:
data = get_tweets_by_username("LibranTechie", 100)
data.head()

In [ ]:
# Fetch the Tweets
keywords = input("Enter keywords, hashtags separated by commas: ")
keywords = list(set(keywords.split(",")))
num_tweets = int(input("Enter number of tweets to be retrieved: "))
data = get_tweets_by_search_term(keywords)

In [ ]:
# Save tweets to pickle file and reload to DF - Avoid API calls
this_file_path = os.path.abspath("__file__")
BASE_DIR = os.path.dirname(this_file_path)
tweet_pickle_file = os.path.join(BASE_DIR, "tweets", "tweet_list.pkl")
data.to_pickle(tweet_pickle_file)


In [ ]:
# Read pickle file
tweets_df = pd.read_pickle(tweet_pickle_file)
tweets_df.head()

In [ ]:
# Primary exploratory analysis
print('Dataset shape:', tweets_df.shape)
print('Dataset columns:', tweets_df.columns)
tweets_df.info()

In [ ]:
# Drop all columns except for Tweet and create a list of all words
tweets_only_df = tweets_df.drop(['UserId', 'TweetId', 'location', 'created'], axis=1)
tweets_only_df.head()

In [ ]:
# Remove http links from tweets
def remove_http_https(text):
    return re.sub(r'http\S+', '', text)

tweets_only_df['link_removed'] = tweets_only_df['tweet'].apply(lambda x: remove_http_https(x))
tweets_only_df.head()

In [ ]:
# Remove punctuations
#string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

tweets_only_df['tweet_punct'] = tweets_only_df['tweet'].apply(lambda x: remove_punct(x))
tweets_only_df.head()

In [ ]:
# Tokenize the tweets
def tokenize(text):
    text = re.split('\W+', text)
    return text

tweets_only_df['tweet_tokenized'] = tweets_only_df['tweet_punct'].apply(lambda x: tokenize(x))
tweets_only_df.head()

In [ ]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopwords]
    return text

tweets_only_df['tweet_nonstop'] = tweets_only_df['tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweets_only_df.head()

In [ ]:
# Stemming and lemmatization
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweets_only_df['tweet_stemmed'] = tweets_only_df['tweet_nonstop'].apply(lambda x: stemming(x))

wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
tweets_only_df['tweet_lemmatized'] = tweets_only_df['tweet_stemmed'].apply(lambda x: lemmatizer(x))
tweets_only_df.head()

In [ ]:
all_final_tweets = ' '.join(word for word in tweets_only_df['tweet_punct'])
fig, ax = plt.subplots(figsize=(30,30))
wordcloud_all_tweets = WordCloud(max_font_size=50, max_words=50, background_color="white").generate(all_final_tweets)
ax.imshow(wordcloud_all_tweets, interpolation='bilinear')
ax.set_title(f'Word Cloud of Top 100 words in 5000 Tweets Mentioning {keywords} ', fontsize=20)
ax.axis("off")

In [ ]:
# Subjectivity of the tweets using TextBlob
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

tweets_df['subjectivity'] = tweets_df['tweet'].apply(get_subjectivity)
tweets_df['polarity'] = tweets_df['tweet'].apply(get_polarity)
tweets_df.head(10)

In [ ]:
# Compute negative, neutral, positive analysis of the tweets
def get_analysis(score):
    if score < 0:
        return 'Negative'
    elif score== 0:
        return 'Neutral'
    else:
        return 'Positive'

tweets_df['analysis'] = tweets_df['polarity'].apply(get_analysis)
tweets_df.head(10)

In [ ]:
def analyse_subjectivity(score):
    if score < 0.5:
        return 'Objective'
    else:
        return 'Subjective'

tweets_df['subjectivity_analysis'] = tweets_df['subjectivity'].apply(analyse_subjectivity)
tweets_df.head(10)

In [ ]:
# Compute the number of positive, negative and neutral tweets
tweets_df['analysis'].value_counts()

In [ ]:
# Plot histogram of the polarity of the tweets
ax0 = tweets_df['polarity'].hist(bins=20, color='blue', edgecolor='black', linewidth=1.2, figsize=(10, 6))
ax0.set_title(f'Histogram of the Polarity of the Tweets - {keywords}', fontsize=12)

In [ ]:
# Plotting the polarity of the tweets
ax1 = tweets_df['analysis'].value_counts().plot(kind='bar', color=['green', 'blue', 'red'], figsize=(10, 8))
ax1.set_title(f'Analysis of Tweets by Search Term: {keywords}', fontsize=20)

In [ ]:
# Pie chart of the polarity of the tweets
ax3 = tweets_df['analysis'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(10, 8))
ax3.set_title(f'% of Positive, Negative and Neutral Tweets - {keywords}', fontsize=10)

In [ ]:
# Pie chart of the subjectivity of the tweets
ax3 = tweets_df['subjectivity_analysis'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(10, 8))
ax3.set_title(f'% of Subjective Tweets - {keywords}', fontsize=10)

In [ ]:
# Plot polarity and subjectivity of the tweets
ax4 = tweets_df.plot(kind='scatter', x='polarity', y='subjectivity', color='blue', figsize=(16,8))
ax4.title.set_text(f'Sentiment Analysis of 5000 Tweets Mentioning - {keywords}')